In [509]:
import pandas as pd
import numpy as np
import psycopg2
import requests
import json

from sklearn.preprocessing import RobustScaler

import tensorflow as tf
from tensorflow.keras.layers import Dense, LSTM 

from keras.optimizers import Adam

pd.options.display.max_columns = None

In [510]:
def new_target_column(dataframe,target_code , shift_back_period):
    prev_target = dataframe[target_code]
    new_target = prev_target[:-shift_back_period]
    first_dates_handling = [0] * shift_back_period
    new_target=np.concatenate((first_dates_handling,new_target))
    return new_target

def make_query(query,endpoint):
    headers = {
        'Accept-Encoding': 'gzip, deflate, br',
        'Content-Type': 'application/json',
        'Accept': 'application/json',
        'Connection': 'keep-alive',
        'DNT': '1'
    }

    response = requests.post(endpoint,json={"query":query},headers=headers)
    return response

def execute_FTP(values,index,columns,pretreatment_attrs=None):
    pretreatment_attrs = json.dumps(pretreatment_attrs)
    query = f"""
        query {{
            featuresTargetsPretreatment(
                values: {values}, 
                index: {index},
                columns: {columns},
                pretreatment_attrs: {pretreatment_attrs}) {{
                    success,
                    error,
                    pretreated_values
                    index,
                    columns,
                    pretreatment_info
                }}
        }}
    """

    ftp_response = make_query(query=query,endpoint='http://172.31.16.204:8000/graphql')
    return ftp_response

def building_data_sequences(data_X, data_Y, timesteps):
    #generate data sequence with timesteps
    X, y_MPNxP = [], [] 
    for i in range(len(data_X) - timesteps + 1):
        sequence = data_X[i:(i + timesteps), :]
        target = data_Y[i + timesteps - 1]
        X.append(sequence)
        y_MPNxP.append(target)
    return np.array(X), [np.array(y_MPNxP)]

def custom_loss_function(attenuated_padding_value):

  def padding_loss_function(y_true, y_pred):

    y_pred = tf.multiply(y_pred, attenuated_padding_value) #this is the multiplication between the predictions and the attenuated_padding_value

    squared_difference = tf.square(y_true - y_pred)

    return tf.reduce_mean(squared_difference, axis=-1) #mse

  return padding_loss_function

def compile_model(input_shape, iteration, model_case_version_main_target_code, optimizer, attenuated_padding_value, output_shape):
    twoexp_nodes_number_layer_1 = 7
    twoexp_nodes_number_layer_2 = 10
    twoexp_nodes_number_layer_3 = 7
    twoexp_nodes_number_layer_4 = 6
    twoexp_nodes_number_layer_5 = 0

    tf.keras.backend.clear_session()
    model= tf.keras.Sequential()
    '''The layers of the model (see case_version_cat Tab)'''
    model.add(LSTM(2**twoexp_nodes_number_layer_1,input_shape=input_shape,return_sequences=True, name=f'prediction_lstm_0_for_iteration_{iteration}'))
    model.add(LSTM(2**twoexp_nodes_number_layer_2, return_sequences=True,name = f'prediction_lstm_1_for_iteration_{iteration}'))
    model.add(LSTM(2**twoexp_nodes_number_layer_3,name = f'prediction_lstm_2_for_iteration_{iteration}'))
    model.add(Dense(2**twoexp_nodes_number_layer_4, name = f'prediction_dense_0_for_iteration_{iteration}'))
    model.add(Dense(output_shape, name = f'prediction_dense_1_for_iteration_{iteration}'))

    model.compile(optimizer = optimizer, loss = custom_loss_function(attenuated_padding_value))

    return model


In [511]:
DBNAME='dyDATA_new'
DATABASE_HOST='database-1.ctzm0hf7fhri.eu-central-1.rds.amazonaws.com'
USER='postgres'
PASSWORD='Proc2023awsrdspostgresql'
DATABASE_PORT=5432

ASSET = 'MSFT'
model_case_version_main_target_code = '5'

use_cols ={
       'cleaned_raw_features_DCP_date_current_period' : 'DCP',
       'calculated_features_DNCP' : 'DNCP',
       'cleaned_raw_features_OPCP_open_price_current_period' : 'OPCP',
       'cleaned_raw_features_HPCP_high_price_current_period' : 'HPCP',
       'cleaned_raw_features_LPCP_low_price_current_period' : 'LPCP',
       'cleaned_raw_features_CPCP_close_price_current_period' : 'CPCP',
       'cleaned_raw_features_ACPCP_adjusted_close_price_current_period' : 'ACPCP',
       'cleaned_raw_features_VTCP_volume_of_transactions_current_period' : 'VTCP',
       'calculated_targets_LPN' +  model_case_version_main_target_code + 'P' : 'LPN' + model_case_version_main_target_code + 'P',
       'calculated_targets_HPN' +  model_case_version_main_target_code + 'P' : 'HPN' + model_case_version_main_target_code + 'P',
       'calculated_targets_MPN' +  model_case_version_main_target_code + 'P' : 'MPN' + model_case_version_main_target_code + 'P',
}

timesteps = 10

In [512]:
connection_params = {
    "host": DATABASE_HOST,
    "port": DATABASE_PORT,
    "user": USER,
    "password": PASSWORD,
    "database": DBNAME
}

conn = psycopg2.connect(**connection_params)

query = f'SELECT * FROM "ASSET_{ASSET}".features_targets_input_view WHERE features_targets_input_view."cleaned_raw_features_environment_PK" = 4'
features_targets_input_view = pd.read_sql(query,conn)
features_targets_input_view = features_targets_input_view[list(use_cols.keys())]
features_targets_input_view.rename(columns=use_cols, inplace=True)

/tmp/ipykernel_1911/3505839591.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  features_targets_input_view = pd.read_sql(query,conn)


In [513]:
features_targets_input_view.tail()

,DCP,DNCP,OPCP,HPCP,LPCP,CPCP,ACPCP,VTCP,LPN5P,HPN5P,MPN5P
6094,2024-03-25,45376.0,425.240,427.41,421.61,422.86,422.86,17860400.0,419.01,427.89,422.86
6095,2024-03-26,45377.0,425.610,425.99,421.35,421.65,421.65,16690800.0,419.01,427.89,422.86
6096,2024-03-27,45378.0,424.440,424.45,419.01,421.43,421.43,16690300.0,419.01,427.89,422.86
6097,2024-03-28,45379.0,420.960,421.87,419.12,420.72,420.72,21861700.0,419.01,427.89,422.86
6098,2024-04-01,45383.0,423.945,427.89,422.22,424.57,424.57,14744117.0,419.01,427.89,422.86


In [514]:
target_codes = features_targets_input_view.columns[-3:]
targets = []

for target_code in target_codes:
    base_target_column_index = features_targets_input_view.columns.get_loc(target_code)
    print(base_target_column_index)
    new_target_index = base_target_column_index
    for i in range(int(model_case_version_main_target_code)):
        new_target_code = target_code[:3] + '-' + str(i+1) + 'P'
        features_targets_input_view.insert(new_target_index+1,new_target_code,new_target_column(features_targets_input_view,target_code,i+1))
        new_target_index = new_target_index + 1
        targets.append(new_target_code)

targets.extend(target_codes)
dataframe = features_targets_input_view.iloc[int(model_case_version_main_target_code):]
#dataframe.set_index('DCP', inplace=True)

8
14
20


In [515]:
dataframe.tail()

,DCP,DNCP,OPCP,HPCP,LPCP,CPCP,ACPCP,VTCP,LPN5P,LPN-1P,LPN-2P,LPN-3P,LPN-4P,LPN-5P,HPN5P,HPN-1P,HPN-2P,HPN-3P,HPN-4P,HPN-5P,MPN5P,MPN-1P,MPN-2P,MPN-3P,MPN-4P,MPN-5P
6094,2024-03-25,45376.0,425.240,427.41,421.61,422.86,422.86,17860400.0,419.01,419.01,419.01,419.01,420.66,415.55,427.89,427.89,429.86,430.82,430.82,430.82,422.86,422.86,423.65,425.61,425.96,425.24
6095,2024-03-26,45377.0,425.610,425.99,421.35,421.65,421.65,16690800.0,419.01,419.01,419.01,419.01,419.01,420.66,427.89,427.89,427.89,429.86,430.82,430.82,422.86,422.86,422.86,423.65,425.61,425.96
6096,2024-03-27,45378.0,424.440,424.45,419.01,421.43,421.43,16690300.0,419.01,419.01,419.01,419.01,419.01,419.01,427.89,427.89,427.89,427.89,429.86,430.82,422.86,422.86,422.86,422.86,423.65,425.61
6097,2024-03-28,45379.0,420.960,421.87,419.12,420.72,420.72,21861700.0,419.01,419.01,419.01,419.01,419.01,419.01,427.89,427.89,427.89,427.89,427.89,429.86,422.86,422.86,422.86,422.86,422.86,423.65
6098,2024-04-01,45383.0,423.945,427.89,422.22,424.57,424.57,14744117.0,419.01,419.01,419.01,419.01,419.01,419.01,427.89,427.89,427.89,427.89,427.89,427.89,422.86,422.86,422.86,422.86,422.86,422.86


### Train test split

In [516]:
train_end_date = '2023-12-29'
train_end_idx = dataframe.index[dataframe['DCP']==train_end_date].values[0]
train_end_new_idx = train_end_idx-int(model_case_version_main_target_code)
train_new_end_date=dataframe.loc[train_end_new_idx,'DCP']

In [517]:
dataframe.set_index('DCP', inplace=True)

In [518]:
train = dataframe.loc[:train_new_end_date]
test = dataframe.loc[train_new_end_date:].drop(train_new_end_date)

In [519]:
train.reset_index(inplace=True)

In [520]:
features = ['DNCP','OPCP','HPCP','LPCP','CPCP','ACPCP']
features_and_targets = features + targets

In [521]:
'''
values = json.dumps(train[features].values.tolist())
index = json.dumps(train.index.to_list())
columns = json.dumps(features)

ftp_response_train = execute_FTP(values=values,index=index,columns=columns).json()
'''

'\nvalues = json.dumps(train[features].values.tolist())\nindex = json.dumps(train.index.to_list())\ncolumns = json.dumps(features)\n\nftp_response_train = execute_FTP(values=values,index=index,columns=columns).json()\n'

In [522]:
train_robust_scaler_features= RobustScaler().fit(train[features].values)
train_robust_scaler_target=RobustScaler().fit(train[targets].values)

In [523]:
train[features] = train_robust_scaler_features.transform(train[features].values)
train[targets] = train_robust_scaler_target.transform(train[targets].values)

/tmp/ipykernel_1911/2842256068.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train[features] = train_robust_scaler_features.transform(train[features].values)
/tmp/ipykernel_1911/2842256068.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train[targets] = train_robust_scaler_target.transform(train[targets].values)


### Training

In [524]:
X_train = train[features].to_numpy("float64")
y_train = train[targets].to_numpy("float64")
# create sequental data
X_train, y_train = building_data_sequences(X_train, y_train, timesteps)

In [525]:
input_shape = ((X_train).shape[1], (X_train).shape[2])
output_shape = y_train[0].shape[1]
optimizer = Adam(learning_rate=0.005)
model = compile_model(
    input_shape,
    1,
    model_case_version_main_target_code,
    optimizer,
    1,
    output_shape
)

In [526]:
history = model.fit(
    X_train,
    y_train,
    batch_size=64,
    epochs=5,
    verbose=2
)

Epoch 1/5


95/95 - 29s - loss: 0.2911 - 29s/epoch - 304ms/step
Epoch 2/5
95/95 - 23s - loss: 0.0126 - 23s/epoch - 247ms/step
Epoch 3/5
95/95 - 23s - loss: 0.0127 - 23s/epoch - 247ms/step
Epoch 4/5
95/95 - 24s - loss: 0.0055 - 24s/epoch - 248ms/step
Epoch 5/5
95/95 - 24s - loss: 0.0024 - 24s/epoch - 248ms/step


In [538]:
train_predictions = model.predict(X_train)

189/189 [==============================] - 7s 39ms/step


In [539]:
train_predictions = train_robust_scaler_target.inverse_transform(train_predictions)

In [540]:
train_predictions.shape

(6018, 18)

In [541]:
train_predictions

array([[ 51.247337,  50.56874 ,  51.627842, ...,  50.79513 ,  54.046055,
         52.447533],
       [ 50.53814 ,  49.868767,  51.007935, ...,  50.246887,  53.240875,
         51.80748 ],
       [ 49.97841 ,  49.189796,  50.364193, ...,  49.642445,  52.4612  ,
         51.142128],
       ...,
       [364.49472 , 359.85052 , 367.9063  , ..., 367.2725  , 366.10437 ,
        368.23523 ],
       [364.86493 , 360.29388 , 368.29874 , ..., 367.6737  , 366.30838 ,
        368.46173 ],
       [365.07617 , 360.50153 , 368.50964 , ..., 367.87146 , 366.53903 ,
        368.7513  ]], dtype=float32)

In [536]:
train_prediction = [a[:1] for a in train_predictions]
train_prediction = np.concatenate(train_prediction)

In [537]:
train_prediction

array([ 51.247337,  50.53814 ,  49.97841 , ..., 364.49472 , 364.86493 ,
       365.07617 ], dtype=float32)